# Save Result in IFC

> Save the results you get as DataFrame in IFC

In [ ]:
# | default_exp _save_result

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

In [ ]:
# | export
from fastcore.basics import patch
import ifcopenshell
import numpy as np
import pandas as pd

In [ ]:
import ifctruss._example
import ifctruss.ifctruss

In [ ]:
dfs = {
    "nodes": ifctruss._example.nodes,
    "bars": ifctruss._example.bars,
    "point_loads": ifctruss._example.point_loads,
}

In [ ]:
model = ifctruss.ifctruss.build(**dfs)

In [ ]:
dfs = ifctruss.ifctruss.view(model)._asdict()

In [ ]:
results = ifctruss.solver.direct_stiffness_method(**dfs)._asdict()

In [ ]:
results["displacments"]

,Node,Displacement_X,Displacement_Y,Displacement_Z
0,0CV4cH5iP6SwCseG64$pib,214.814815,0.0,-195.833333


In [ ]:
results["forces"]

,Node,Force_X,Force_Y,Force_Z
1,3J7tRX7yb17wDVifRR_qgU,0.000000,0.0,65277.777778
2,34w8f6zkPApPZfWTKgvbUK,-53703.703704,0.0,0.000000
3,306azRYJT07wE6fH29mNtN,-46296.296296,0.0,34722.222222


In [ ]:
results["normal_forces"]

,Bar,Normal_force,Type_of_normal_force
0,1JhZi2bnLCkA7QZzjWN_Cu,-65277.777778,Compressive force
1,2Zy_gy04j21OdzTrdIUh5z,53703.703704,Tensile force
2,0c0bjRSZPEkAptR1IgVoVV,57870.370370,Tensile force


In [ ]:
ifc_structural_analysis_model = model.by_type("IfcStructuralAnalysisModel")[
    0
]
structural_analysis_model = ifc_structural_analysis_model.GlobalId
load_group = ifc_structural_analysis_model.LoadedBy[0].GlobalId

In [ ]:
# | export
class SaveResult:
    def __init__(
        self,
        *,
        model: ifcopenshell.file,
        structural_analysis_model,
        load_group,
        theory_type,
        is_linear: bool,
        displacments,
        forces,
        normal_forces
    ):
        self.ifc_model = model
        self.displacment = displacments
        self.force = forces
        self.normal_force = normal_forces

        self.structural_analysis_model = self.ifc_model.by_guid(
            structural_analysis_model
        )
        self.load_group = self.ifc_model.by_guid(load_group)
        self.theory_type = theory_type
        self.is_linear = is_linear

In [ ]:
save_result_object = SaveResult(
    model=model,
    structural_analysis_model=structural_analysis_model,
    load_group=load_group,
    **results
)

In [ ]:
# | export
@patch
def create_structural_result_group(
    self: SaveResult,
):
    self.structural_result_group = (
        self.ifc_model.createIfcStructuralResultGroup(
            ifcopenshell.guid.new(),
            None,
            None,
            None,
            None,
            self.theory_type,
            self.load_group,
            self.is_linear,
        )
    )

    if self.structural_analysis_model.HasResults is not None:
        self.structural_analysis_model.HasResults += (
            self.structural_result_group,
        )
    elif self.structural_analysis_model.HasResults is None:
        self.structural_analysis_model.HasResults = (
            self.structural_result_group,
        )

    self.ifc_results = []

In [ ]:
save_result_object.create_structural_result_group()

In [ ]:
save_result_object.structural_analysis_model.HasResults

(#77=IfcStructuralResultGroup('3zAO6xs_54VAEKNOI84cBw',$,$,$,$,.FIRST_ORDER_THEORY.,#66,.T.),)

In [ ]:
model.by_type("IfcStructuralResultGroup")

[#77=IfcStructuralResultGroup('3zAO6xs_54VAEKNOI84cBw',$,$,$,$,.FIRST_ORDER_THEORY.,#66,.T.)]

In [ ]:
save_result_object.load_group.SourceOfResultGroup

(#77=IfcStructuralResultGroup('3zAO6xs_54VAEKNOI84cBw',$,$,$,$,.FIRST_ORDER_THEORY.,#66,.T.),)

In [ ]:
save_result_object.ifc_results

[]

In [ ]:
# | export
@patch
def create_displacment_for_one_node(self: SaveResult, row):
    node = row["Node"]
    displacement_x = row["Displacement_X"]
    displacement_y = row["Displacement_Y"]
    displacement_z = row["Displacement_Z"]

    name = "Result_Displacement"
    structural_load_single_displacement = (
        self.ifc_model.createIfcStructuralLoadSingleDisplacement(
            name,
            displacement_x,
            displacement_y,
            displacement_z,
            None,
            None,
            None,
        )
    )

    global_or_local = "GLOBAL_COORDS"
    structural_point_reaction = (
        self.ifc_model.createIfcStructuralPointReaction(
            ifcopenshell.guid.new(),
            None,
            name,
            None,
            None,
            None,
            None,
            structural_load_single_displacement,
            global_or_local,
        )
    )

    self.ifc_model.createIfcRelConnectsStructuralActivity(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        self.ifc_model.by_guid(node),
        structural_point_reaction,
    )

    return structural_point_reaction

In [ ]:
# | export
@patch
def create_displacments(
    self: SaveResult,
):
    self.ifc_results += (
        self.displacment.apply(self.create_displacment_for_one_node, axis=1)
    ).tolist()

In [ ]:
save_result_object.displacment

,Node,Displacement_X,Displacement_Y,Displacement_Z
0,0CV4cH5iP6SwCseG64$pib,214.814815,0.0,-195.833333


In [ ]:
save_result_object.create_displacments()

In [ ]:
model.by_type("IfcStructuralLoadSingleDisplacement")

[#78=IfcStructuralLoadSingleDisplacement('Result_Displacement',214.814814814815,0.,-195.833333333333,$,$,$)]

In [ ]:
model.by_type("IfcStructuralPointReaction")

[#79=IfcStructuralPointReaction('2YxV$$kHbFN9OjI4Ad5bnm',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.)]

In [ ]:
save_result_object.ifc_results

[#79=IfcStructuralPointReaction('2YxV$$kHbFN9OjI4Ad5bnm',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.)]

In [ ]:
# | export
@patch
def create_force_for_one_node(self: SaveResult, row):
    node = row["Node"]
    force_x = row["Force_X"]
    force_y = row["Force_Y"]
    force_z = row["Force_Z"]
    name = "Result_Force"
    structural_load_single_force = (
        self.ifc_model.createIfcStructuralLoadSingleForce(
            name, force_x, force_y, force_z, None, None, None
        )
    )

    global_or_local = "GLOBAL_COORDS"

    structural_point_reaction = (
        self.ifc_model.createIfcStructuralPointReaction(
            ifcopenshell.guid.new(),
            None,
            name,
            None,
            None,
            None,
            None,
            structural_load_single_force,
            global_or_local,
        )
    )

    self.ifc_model.createIfcRelConnectsStructuralActivity(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        self.ifc_model.by_guid(node),
        structural_point_reaction,
    )

    return structural_point_reaction

In [ ]:
# | export
@patch
def create_forces(
    self: SaveResult,
):
    self.ifc_results += (
        self.force.apply(self.create_force_for_one_node, axis=1)
    ).tolist()

In [ ]:
save_result_object.force

,Node,Force_X,Force_Y,Force_Z
1,3J7tRX7yb17wDVifRR_qgU,0.000000,0.0,65277.777778
2,34w8f6zkPApPZfWTKgvbUK,-53703.703704,0.0,0.000000
3,306azRYJT07wE6fH29mNtN,-46296.296296,0.0,34722.222222


In [ ]:
save_result_object.create_forces()

In [ ]:
model.by_type("IfcStructuralLoadSingleForce")

[#63=IfcStructuralLoadSingleForce('Point Load 1',100000.,0.,-100000.,$,$,$),
 #81=IfcStructuralLoadSingleForce('Result_Force',0.,0.,65277.7777777778,$,$,$),
 #84=IfcStructuralLoadSingleForce('Result_Force',-53703.7037037037,0.,0.,$,$,$),
 #87=IfcStructuralLoadSingleForce('Result_Force',-46296.2962962963,0.,34722.2222222222,$,$,$)]

In [ ]:
model.by_type("IfcStructuralPointReaction")

[#79=IfcStructuralPointReaction('2YxV$$kHbFN9OjI4Ad5bnm',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.),
 #82=IfcStructuralPointReaction('0hGQ$$VkT0dg1q5C$$xjwi',$,'Result_Force',$,$,$,$,#81,.GLOBAL_COORDS.),
 #85=IfcStructuralPointReaction('0wdM6c9Q14UBzvwCKWkc_Y',$,'Result_Force',$,$,$,$,#84,.GLOBAL_COORDS.),
 #88=IfcStructuralPointReaction('3FjOmBDLzECejkbWUuvPTd',$,'Result_Force',$,$,$,$,#87,.GLOBAL_COORDS.)]

In [ ]:
save_result_object.ifc_results

[#79=IfcStructuralPointReaction('2YxV$$kHbFN9OjI4Ad5bnm',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.),
 #82=IfcStructuralPointReaction('0hGQ$$VkT0dg1q5C$$xjwi',$,'Result_Force',$,$,$,$,#81,.GLOBAL_COORDS.),
 #85=IfcStructuralPointReaction('0wdM6c9Q14UBzvwCKWkc_Y',$,'Result_Force',$,$,$,$,#84,.GLOBAL_COORDS.),
 #88=IfcStructuralPointReaction('3FjOmBDLzECejkbWUuvPTd',$,'Result_Force',$,$,$,$,#87,.GLOBAL_COORDS.)]

$$l_x = x_2 - x_1, \quad l_y = y_2 - y_1, \quad l_z = z_2 - z_1$$

In [ ]:
# | export
@patch
def calculate_differences(self: SaveResult, x_1, y_1, z_1, x_2, y_2, z_2):
    # copied from DirectStiffnessMethod class
    l_x = x_2 - x_1
    l_y = y_2 - y_1
    l_z = z_2 - z_1
    return l_x, l_y, l_z

$$l = \sqrt{l_x^2 + l_y^2 + l_z^2}$$

In [ ]:
# | export
@patch
def calculate_distance(self: SaveResult, l_x, l_y, l_z):
    # copied from DirectStiffnessMethod class
    vector = np.array([l_x, l_y, l_z])
    l = np.linalg.norm(vector)
    return l

In [ ]:
# | export
@patch
def get_coordinates(self: SaveResult, node):
    # copied from View class and modified
    coordinates = node.VertexGeometry.Coordinates
    return coordinates

In [ ]:
# | export
@patch
def get_start_node(self: SaveResult, bar):
    # copied from View class and modified
    start_node = bar.Representation.Representations[0].Items[0].EdgeStart
    return start_node

In [ ]:
# | export
@patch
def get_end_node(self: SaveResult, bar):
    # copied from View class and modified
    end_node = bar.Representation.Representations[0].Items[0].EdgeEnd
    return end_node

$$F_2=-F_1=N$$

In [ ]:
# | export
@patch
def get_length(self: SaveResult, row):
    bar = row["Bar"]
    normal_force = row["Normal_force"]

    ifc_bar = self.ifc_model.by_guid(bar)

    start_node = self.get_start_node(ifc_bar)
    end_node = self.get_end_node(ifc_bar)

    start_node_coordinates = self.get_coordinates(start_node)
    end_node_coordinates = self.get_coordinates(end_node)

    l_x, l_y, l_z = self.calculate_differences(
        *start_node_coordinates, *end_node_coordinates
    )

    length = self.calculate_distance(l_x, l_y, l_z)

    return length

In [ ]:
# | export
@patch
def create_normal_force_for_one_bar(self: SaveResult, row):
    bar = row["Bar"]
    normal_force = row["Normal_force"]
    name = "Result_Normal_Force"
    length = self.get_length(row)

    structural_load_single_force_tail = (
        self.ifc_model.createIfcStructuralLoadSingleForce(
            Name="Tail",
            ForceX=normal_force,
        )
    )

    structural_load_single_force_head = (
        self.ifc_model.createIfcStructuralLoadSingleForce(
            Name="Head",
            ForceX=-normal_force,
        )
    )

    structural_load_configuration = (
        self.ifc_model.createIfcStructuralLoadConfiguration(
            Name="Member End Reactions",
            Values=(
                structural_load_single_force_head,
                structural_load_single_force_tail,
            ),
            Locations=[
                [0.0],
                [float(length)],
            ],
        )
    )
    global_or_local = "LOCAL_COORDS"
    predefined_type = "DISCRETE"
    structural_curve_reaction = (
        self.ifc_model.createIfcStructuralCurveReaction(
            ifcopenshell.guid.new(),
            None,
            name,
            None,
            None,
            None,
            None,
            structural_load_configuration,
            global_or_local,
            predefined_type,
        )
    )

    self.ifc_model.createIfcRelConnectsStructuralActivity(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        self.ifc_model.by_guid(bar),
        structural_curve_reaction,
    )

    return structural_curve_reaction

In [ ]:
# | export
@patch
def create_normal_forces(
    self: SaveResult,
):
    self.ifc_results += (
        self.normal_force.apply(
            self.create_normal_force_for_one_bar, axis=1
        )
    ).tolist()

In [ ]:
save_result_object.create_normal_forces()

In [ ]:
save_result_object.ifc_results

[#79=IfcStructuralPointReaction('2YxV$$kHbFN9OjI4Ad5bnm',$,'Result_Displacement',$,$,$,$,#78,.GLOBAL_COORDS.),
 #82=IfcStructuralPointReaction('0hGQ$$VkT0dg1q5C$$xjwi',$,'Result_Force',$,$,$,$,#81,.GLOBAL_COORDS.),
 #85=IfcStructuralPointReaction('0wdM6c9Q14UBzvwCKWkc_Y',$,'Result_Force',$,$,$,$,#84,.GLOBAL_COORDS.),
 #88=IfcStructuralPointReaction('3FjOmBDLzECejkbWUuvPTd',$,'Result_Force',$,$,$,$,#87,.GLOBAL_COORDS.),
 #93=IfcStructuralCurveReaction('0lfG6QPhr6lwRHGBDC6$bJ',$,'Result_Normal_Force',$,$,$,$,#92,.LOCAL_COORDS.,.DISCRETE.),
 #98=IfcStructuralCurveReaction('0uBFrc5ar2PgyVZ9ZEzDBP',$,'Result_Normal_Force',$,$,$,$,#97,.LOCAL_COORDS.,.DISCRETE.),
 #103=IfcStructuralCurveReaction('2L6k$YfKn4QPtGSt_L7cJX',$,'Result_Normal_Force',$,$,$,$,#102,.LOCAL_COORDS.,.DISCRETE.)]

In [ ]:
model.by_type("IfcStructuralCurveReaction")

[#93=IfcStructuralCurveReaction('0lfG6QPhr6lwRHGBDC6$bJ',$,'Result_Normal_Force',$,$,$,$,#92,.LOCAL_COORDS.,.DISCRETE.),
 #98=IfcStructuralCurveReaction('0uBFrc5ar2PgyVZ9ZEzDBP',$,'Result_Normal_Force',$,$,$,$,#97,.LOCAL_COORDS.,.DISCRETE.),
 #103=IfcStructuralCurveReaction('2L6k$YfKn4QPtGSt_L7cJX',$,'Result_Normal_Force',$,$,$,$,#102,.LOCAL_COORDS.,.DISCRETE.)]

In [ ]:
model.by_type("IfcStructuralLoadConfiguration")

[#92=IfcStructuralLoadConfiguration('Member End Reactions',(#91,#90),((0.),(3000.))),
 #97=IfcStructuralLoadConfiguration('Member End Reactions',(#96,#95),((0.),(4000.))),
 #102=IfcStructuralLoadConfiguration('Member End Reactions',(#101,#100),((0.),(5000.)))]

In [ ]:
[
    member.Values
    for member in model.by_type("IfcStructuralLoadConfiguration")
]

[(#91=IfcStructuralLoadSingleForce('Head',65277.7777777778,$,$,$,$,$),
  #90=IfcStructuralLoadSingleForce('Tail',-65277.7777777778,$,$,$,$,$)),
 (#96=IfcStructuralLoadSingleForce('Head',-53703.7037037037,$,$,$,$,$),
  #95=IfcStructuralLoadSingleForce('Tail',53703.7037037037,$,$,$,$,$)),
 (#101=IfcStructuralLoadSingleForce('Head',-57870.3703703704,$,$,$,$,$),
  #100=IfcStructuralLoadSingleForce('Tail',57870.3703703704,$,$,$,$,$))]

In [ ]:
# | export
@patch
def assign_to_the_result_group(
    self: SaveResult,
):
    self.ifc_model.createIfcRelAssignsToGroup(
        ifcopenshell.guid.new(),
        None,
        None,
        None,
        self.ifc_results,
        None,
        self.structural_result_group,
    )

In [ ]:
save_result_object.assign_to_the_result_group()

In [ ]:
save_result_object.ifc_model.write("truss_result.ifc")

In [ ]:
import ifcopenshell.validate
from rich import print

In [ ]:
json_logger = ifcopenshell.validate.json_logger()
ifcopenshell.validate.validate(save_result_object.ifc_model, json_logger)
json_list = json_logger.statements
if json_list:
    for i in json_list:
        print(i)

An empty list will evaluate to False.

In [ ]:
assert not json_list

In [ ]:
# | hide
nbdev.nbdev_export()